# Create a manifest file

The *manifest.txt* file created here is intended to be used with [gdc-client](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)

Please refer to [gdc API](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/) for reference about query done here

In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,requests,json -a Filippo_Valle -g -r -b -w

Filippo_Valle 
last updated: Tue Jun 30 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
requests 2.23.0
json 2.0.9

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 728c5fcb2ae65fb2d06e3b61b8b167d388b13f95
Git repo   : git@github.com:fvalle1/topics.git
Git branch : develop
watermark 2.0.2


In [2]:
import requests
import json
import pandas as pd

In [9]:
# use file endopint
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.demographic.vital_status",
    "cases.demographic.days_to_birth",
    "cases.demographic.days_to_death",
    "cases.demographic.gender",
    "cases.diagnoses.primary_diagnosis",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.last_known_disease_status",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.days_to_last_follow_up",
    "cases.diagnoses.age_at_diagnosis",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.exposures.years_smoked",
    "cases.exposures.cigarettes_per_day",
    "cases.samples.longest_dimension",
    "samples.portions.analytes.aliquots.submitter_id",
    #"cases.project.project_id"
]

fields = ','.join(fields)

Filter gene expression data obtained with *RNA-Seq HTSeq* in txt format either from **BRCA** or **LUAD** and **LUSC**

Don't forget to select the correct filter for *cases.project.project_id*

In [30]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-LUAD', 'TCGA-LUSC'] #<-- uncomment this line for Lung
            #"value": ["TCGA-BRCA"] #<-- uncomment this line for Breast
            }
        }
        
    ]
}

In [31]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [32]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [33]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

# Files

Write a file with all metadata, useful in next analyses

In [34]:
params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [35]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(3)

,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.morphology,cases.0.diagnoses.0.primary_diagnosis,cases.0.diagnoses.0.tissue_or_organ_of_origin,cases.0.diagnoses.0.tumor_grade,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.disease_type,cases.0.project.primary_site,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
file_name,,,,,,,,,,,,,,,,,,
1142583c-2ca0-4f95-a94b-6f1a35ab1580.FPKM.txt.gz,-19088.0,444.0,male,Dead,19088.0,not reported,8140/3,"Adenocarcinoma, NOS","Lower lobe, lung",not reported,stage iiia,1.972603,NaN,Lung Adenocarcinoma,Lung,2.5,TCGA-86-8359-01A-11R-2326-07,TCGA-86-8359
ab51cccb-b5df-434a-a5a4-32368ccf0b24.FPKM.txt.gz,-18856.0,761.0,male,Dead,18856.0,not reported,8140/3,"Adenocarcinoma, NOS","Upper lobe, lung",not reported,stage ib,3.287671,30.0,Lung Adenocarcinoma,Lung,NaN,TCGA-44-2668-01B-02R-A277-07,TCGA-44-2668
fee154e0-ff94-402d-91e3-6c3d458bee00.FPKM.txt.gz,-24903.0,NaN,female,Alive,24903.0,not reported,8253/3,"Bronchio-alveolar carcinoma, mucinous","Upper lobe, lung",not reported,stage ia,5.205479,38.0,Lung Adenocarcinoma,Lung,NaN,TCGA-S2-AA1A-01A-12R-A39D-07,TCGA-S2-AA1A


In [36]:
# save file to dat
df_files.to_csv("files.dat", header=True)